In [1]:
# Cell 0 - Setup (Colab)
# Run in Google Colab: mounts drive, installs timm
from google.colab import drive
drive.mount('/content/drive')

# Install packages if needed
!pip install -q timm scikit-learn pandas tqdm


Mounted at /content/drive


In [2]:
# Cell 1 - imports & config
import os, glob, random, time
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from tqdm.auto import tqdm

# Config - EDIT if needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
project_dir = "/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection"  # <- edit if needed
train_image_root = f"{project_dir}/data/sneakers"
test_image_root = f"{project_dir}/data/test"
BATCH_SIZE = 24
IMG_SIZE = 224
NUM_EPOCHS = 12
LR = 2e-4
WEIGHT_DECAY = 1e-4
BACKBONE = 'vit_base_patch16_224'   # timm model
FREEZE_BACKBONE = False
BRAND_EMBED_DIM = 64
OUT_DIM = 512
USE_CONTRASTIVE = True
CONTRASTIVE_TEMPERATURE = 0.2
AUX_BRAND_LOSS = True
SAVE_PATH = "/content/best_model.pth"

print("Device:", DEVICE)


Device: cuda


In [3]:
print(DEVICE)

cuda


In [4]:
# Cell 2 - read CSVs (per-brand), merge, build image_path column
csv_files = sorted([p for p in glob.glob(f"{project_dir}/*.csv") if not os.path.basename(p).lower().startswith("test")])
print("CSV files (excluded test.csv):", csv_files)

dfs = []
for f in csv_files:
    df = pd.read_csv(f)
    # ensure required columns exist
    assert 'Image Name' in df.columns and 'Brand' in df.columns and 'Price' in df.columns and 'Authentic' in df.columns, \
        f"CSV {f} missing required columns"
    dfs.append(df)

full_df = pd.concat(dfs, ignore_index=True)
full_df = full_df.dropna(subset=['Image Name', 'Brand']).reset_index(drop=True)
print("Total merged rows:", len(full_df))

# path builder for train/val images using authentic/fake subfolders
def build_path(row):
    brand = str(row["Brand"]).lower()
    img = row["Image Name"]
    label = int(row["Authentic"])  # 1 authentic, 0 fake
    subfolder = "authentic" if label == 1 else "fake"
    return os.path.join(train_image_root, subfolder, brand, str(img))

full_df['image_path'] = full_df.apply(build_path, axis=1)

# quick check: drop rows whose files do not exist (optional but helpful)
missing_mask = ~full_df['image_path'].apply(os.path.exists)
if missing_mask.any():
    print(f"Warning: {missing_mask.sum()} image paths missing on disk. Showing examples:")
    display(full_df[missing_mask].head(5))
    # Optionally drop missing rows:
    # full_df = full_df[~missing_mask].reset_index(drop=True)

# 80-20 train/val split stratified by Brand
train_df, val_df = train_test_split(full_df, test_size=0.2, stratify=full_df['Brand'], random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print("Train / Val sizes:", len(train_df), len(val_df))


CSV files (excluded test.csv): ['/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/Counterfeit_product_data - Adidas.csv', '/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/Counterfeit_product_data - Jordan .csv', '/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/Counterfeit_product_data - Nike.csv', '/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/Counterfeit_product_data - Puma.csv', '/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/Counterfeit_product_data - Reebok.csv']
Total merged rows: 258
Train / Val sizes: 206 52


In [5]:
full_df['image_path'][0]

'/content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/data/sneakers/fake/adidas/1.png'

In [6]:
# Cell 3 - transforms
train_transforms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(8),
    T.ColorJitter(0.08,0.08,0.08,0.02),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
val_transforms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])


In [7]:
# Cell 4 - fit brand encoder and price scaler on train only
brand_encoder = LabelEncoder()
brand_encoder.fit(train_df['Brand'].astype(str).fillna("unknown"))
num_brands = len(brand_encoder.classes_)
print("Num brands:", num_brands, brand_encoder.classes_[:10])

price_scaler = StandardScaler()
train_prices = np.log1p(train_df['Price'].astype(float).fillna(0.0).values.reshape(-1,1))
price_scaler.fit(train_prices)


Num brands: 5 ['Adidas' 'Jordan' 'Nike' 'Puma' 'Reebok']


StandardScaler()

In [8]:
# Cell 5 - Dataset
class SneakerDataset(Dataset):
    def __init__(self, df, brand_encoder, price_scaler, transforms=None, is_train=False, image_root_override=None):
        self.df = df.reset_index(drop=True)
        self.brand_encoder = brand_encoder
        self.price_scaler = price_scaler
        self.transforms = transforms
        self.is_train = is_train
        self.image_root_override = image_root_override  # if you want to override path building for test

        # precompute brand indices and scaled prices
        self.brand_idxs = self.brand_encoder.transform(self.df['Brand'].astype(str).fillna("unknown"))
        prices = np.log1p(self.df['Price'].astype(float).fillna(0.0).values.reshape(-1,1))
        self.prices_scaled = self.price_scaler.transform(prices).squeeze()

        # Handle labels if present
        if 'Authentic' in self.df.columns:
            self.labels = self.df['Authentic'].astype(int).values
        else:
            self.labels = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # use existing image_path if present, else build from pattern
        if 'image_path' in self.df.columns and pd.notna(row['image_path']):
            img_path = row['image_path']
        else:
            # fallback logic
            brand = str(row['Brand']).lower()
            img = row['Image Name']

            if self.image_root_override:
                # FIX: User specified flat structure in test folder -> root/image_name
                img_path = os.path.join(self.image_root_override, str(img))
            else:
                # Default train structure -> root/brand/image_name
                img_path = os.path.join(test_image_root, brand, str(img))

        # open image
        img = Image.open(img_path).convert('RGB')
        if self.transforms:
            img = self.transforms(img)

        brand_idx = int(self.brand_idxs[idx])
        price = float(self.prices_scaled[idx])

        sample = {'image': img, 'brand': torch.tensor(brand_idx, dtype=torch.long), 'price': torch.tensor(price, dtype=torch.float32)}
        if self.labels is not None:
            sample['label'] = torch.tensor(self.labels[idx], dtype=torch.float32)
        return sample

In [9]:
# Cell 6 - datasets & loaders
train_ds = SneakerDataset(train_df, brand_encoder, price_scaler, transforms=train_transforms, is_train=True)
val_ds   = SneakerDataset(val_df, brand_encoder, price_scaler, transforms=val_transforms, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)


In [10]:
# Cell 7 - Loss function

auth_criterion = nn.BCEWithLogitsLoss()
brand_criterion = nn.CrossEntropyLoss()

# contrastive loss
def supervised_contrastive_loss(emb, labels, temperature=0.07):
    emb = F.normalize(emb, dim=1)
    logits = torch.matmul(emb, emb.t()) / temperature
    labels = labels.view(-1,1)
    mask_pos = (labels == labels.t()).float()
    diag = torch.eye(logits.size(0), device=logits.device)
    logits_masked = logits - 1e9 * diag
    exp_logits = torch.exp(logits_masked)
    denom = exp_logits.sum(dim=1, keepdim=True)
    pos_exp = (exp_logits * mask_pos).sum(dim=1, keepdim=True)
    loss = -torch.log((pos_exp + 1e-12) / (denom + 1e-12))
    return loss.mean()

In [11]:
# Cell 8 - training & validation
def train_one_epoch(model, loader, optimizer, epoch):
    model.train()
    running_loss = 0.0
    pbar = tqdm(loader, desc=f"Train {epoch}")
    for batch in pbar:
        imgs = batch['image'].to(DEVICE)
        brands = batch['brand'].to(DEVICE)
        prices = batch['price'].to(DEVICE)
        labels = batch.get('label', None)
        if labels is not None:
            labels = labels.to(DEVICE)
        optimizer.zero_grad()
        auth_logits, brand_logits, img_emb, contrast_vec = model(imgs, brands, prices)
        if labels is None:
            continue
        auth_loss = auth_criterion(auth_logits, labels)
        loss = auth_loss
        if AUX_BRAND_LOSS:
            brand_loss = brand_criterion(brand_logits, brands)
            loss = loss + 0.5 * brand_loss
        else:
            brand_loss = torch.tensor(0.0, device=DEVICE)
        if USE_CONTRASTIVE:
            contr_loss = supervised_contrastive_loss(contrast_vec, brands)
            loss = loss + 0.1 * contr_loss
        else:
            contr_loss = torch.tensor(0.0, device=DEVICE)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        pbar.set_postfix({'loss': running_loss / ((pbar.n+1)*BATCH_SIZE)})
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

def validate(model, loader):
    model.eval()
    y_true, y_pred_probs = [], []
    losses = []
    with torch.no_grad():
        for batch in loader:
            imgs = batch['image'].to(DEVICE)
            brands = batch['brand'].to(DEVICE)
            prices = batch['price'].to(DEVICE)
            labels = batch.get('label', None)
            if labels is not None:
                labels = labels.to(DEVICE)
            auth_logits, brand_logits, img_emb, contrast_vec = model(imgs, brands, prices)
            if labels is None:
                continue
            loss = auth_criterion(auth_logits, labels)
            losses.append(loss.item() * imgs.size(0))
            probs = torch.sigmoid(auth_logits).cpu().numpy()
            y_pred_probs.extend(probs.tolist())
            y_true.extend(labels.cpu().numpy().astype(int).tolist())
    avg_loss = np.sum(losses) / len(loader.dataset) if losses else 0.0
    preds = [1 if p>=0.5 else 0 for p in y_pred_probs]
    acc = accuracy_score(y_true, preds) if len(y_true)>0 else 0.0
    return avg_loss, acc


In [12]:
# Cell 9 - training run

# --- FIX: Refined Model Architecture ---
class ViTMultiHeadFixed(nn.Module):
    def __init__(self, backbone_name=BACKBONE, pretrained=True, freeze_backbone=FREEZE_BACKBONE, out_dim=OUT_DIM, brand_embed_dim=BRAND_EMBED_DIM, num_brands=num_brands):
        super().__init__()
        # num_classes=0 + global_pool='avg' -> model(x) returns 1D vector per image
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0, global_pool='avg')
        feat_dim = self.backbone.num_features
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False
        self.img_proj = nn.Sequential(nn.Linear(feat_dim, out_dim), nn.ReLU(), nn.LayerNorm(out_dim))
        self.brand_emb = nn.Embedding(num_brands, brand_embed_dim)

        # A. Increased price_proj to 64 dimensions
        self.price_proj = nn.Sequential(nn.Linear(1, 64), nn.ReLU(), nn.LayerNorm(64))

        combined = out_dim + brand_embed_dim + 64

        # A. Increased hidden dims in auth_head to 512 -> 128 -> 1
        # A. Added more dropout for regularization
        self.auth_head = nn.Sequential(
            nn.Linear(combined, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

        # A. Added brand classification head with higher hidden dim (e.g. 256)
        self.brand_head = nn.Sequential(
            nn.Linear(out_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_brands)
        )

        self.proj_for_contrast = nn.Sequential(nn.Linear(out_dim, out_dim), nn.ReLU(), nn.LayerNorm(out_dim))

    def forward(self, image, brand_idx, price):
        # FIX: use self.backbone(image) instead of forward_features to get pooled features
        feat = self.backbone(image)
        img_emb = self.img_proj(feat)
        brand_vec = self.brand_emb(brand_idx)
        price_vec = self.price_proj(price.unsqueeze(1))

        combined = torch.cat([img_emb, brand_vec, price_vec], dim=1)
        auth_logits = self.auth_head(combined).squeeze(1)
        brand_logits = self.brand_head(img_emb)
        contrast_vec = self.proj_for_contrast(img_emb)
        return auth_logits, brand_logits, img_emb, contrast_vec

# Re-init model and optimizer with the refined class
print("Re-initializing refined model...")
model = ViTMultiHeadFixed().to(DEVICE)

# C. Optimizer and Scheduler: CosineAnnealingLR
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
# ----------------------------------------------------------

best_val_acc = 0.0
for epoch in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    train_loss = train_one_epoch(model, train_loader, optimizer, epoch)
    val_loss, val_acc = validate(model, val_loader)
    scheduler.step()
    print(f"Epoch {epoch}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}, Val Acc = {val_acc:.4f}, Time = {time.time()-t0:.1f}s")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'brand_encoder_classes': brand_encoder.classes_,
            'price_scaler_mean': price_scaler.mean_,
            'price_scaler_scale': price_scaler.scale_
        }, SAVE_PATH)
        print("Saved best model to", SAVE_PATH)
print("Best val acc:", best_val_acc)

Re-initializing refined model...


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Train 1:   0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in by

Epoch 1: Train Loss = 1.627037, Val Loss = 0.584157, Val Acc = 0.7500, Time = 22.5s
Saved best model to /content/best_model.pth


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cd42352a3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cd42352a3e0>^
^Traceback (most recent call last):
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
^    ^self._shutdown_workers()^
^^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    ^^if w.is_alive():^
 ^ ^

Epoch 11: Train Loss = 1.042186, Val Loss = 0.287250, Val Acc = 0.8462, Time = 4.1s


Train 12:   0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 12: Train Loss = 0.998817, Val Loss = 0.286002, Val Acc = 0.8462, Time = 3.3s
Best val acc: 0.8653846153846154


In [18]:
# Cell 10 - Test Config & Path Check
TEST_CSV = os.path.join(project_dir, "test.csv")
# User specified path: project_dir + data/sneakers/test
TEST_IMAGE_ROOT = os.path.join(project_dir, "data/sneakers/test")

print(f"Test CSV Path: {TEST_CSV}")
print(f"Test Image Root: {TEST_IMAGE_ROOT}")

if os.path.exists(TEST_CSV):
    # auto-detect separator (e.g. tab or comma) and strip whitespace from names
    test_df_check = pd.read_csv(TEST_CSV, sep=None, engine='python')
    test_df_check.columns = [c.strip() for c in test_df_check.columns]
    print("Test CSV Columns found:", test_df_check.columns.tolist())
    print("First row example:")
    display(test_df_check.head(1))
else:
    print("Warning: Test CSV not found at specified path.")

if not os.path.exists(TEST_IMAGE_ROOT):
    print(f"Warning: Test image directory not found at {TEST_IMAGE_ROOT}")

Test CSV Path: /content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/test.csv
Test Image Root: /content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/data/sneakers/test
Test CSV Columns found: ['sln', 'Image Name', 'Brand', 'Price', 'Authentic']
First row example:


,sln,Image Name,Brand,Price,Authentic
0,5,5.jpg,Adidas,23,0


In [16]:
# Cell 11 - Test loader function
def load_test_dataset(test_csv_path):
    # Load with flexible separator and clean columns
    test_df = pd.read_csv(test_csv_path, sep=None, engine='python')
    test_df.columns = [c.strip() for c in test_df.columns]

    # Explicitly define test root: .../data/sneakers/test
    test_root = os.path.join(project_dir, "data/sneakers/test")

    print(f"Loading test images from: {test_root}")

    # Initialize Dataset with the override root (flat structure)
    test_ds = SneakerDataset(
        test_df,
        brand_encoder=brand_encoder,
        price_scaler=price_scaler,
        transforms=val_transforms,
        is_train=False,
        image_root_override=test_root
    )

    # Fix: num_workers=0 for Colab safety
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)
    return test_df, test_loader

In [17]:
# Cell 12 - Evaluate on multiple test images
def evaluate_test_csv(test_csv_path):
    test_df, test_loader = load_test_dataset(test_csv_path)

    model.eval()
    preds = []
    probs = []
    targets = []

    print("Starting inference on test set...")
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            img = batch["image"].to(DEVICE)
            brand = batch["brand"].to(DEVICE)
            price = batch["price"].to(DEVICE)

            # Collect ground truth if available
            if 'label' in batch:
                targets.extend(batch['label'].cpu().numpy().tolist())

            # Fix: Model returns 4 values, extract auth_logits (index 0)
            outputs = model(img, brand, price)
            auth_logits = outputs[0]

            p = torch.sigmoid(auth_logits)

            probs.extend(p.cpu().numpy().tolist())
            preds.extend((p > 0.5).int().cpu().numpy().tolist())

    # Append predictions to dataframe
    test_df["Pred_Prob"] = probs
    test_df["Pred_Label"] = preds

    # Calculate metrics if we have targets (y_test)
    if len(targets) > 0:
        acc = accuracy_score(targets, preds)
        print(f"\nTest Accuracy: {acc:.4f}")
        try:
            auc = roc_auc_score(targets, probs)
            print(f"Test AUC: {auc:.4f}")
        except:
            print("Could not calc AUC (maybe only 1 class present)")

    return test_df

In [22]:
# Cell 13 - Run Test Evaluation
# TEST_CSV = f"{project_dir}/test.csv"

if os.path.exists(TEST_CSV):
    results = evaluate_test_csv(TEST_CSV)

    print("\n===== SAMPLE PREDICTIONS =====")
    # Show relevant columns
    cols_to_show = ['Image Name', 'Brand', 'Authentic', 'Pred_Prob', 'Pred_Label']
    cols_to_show = [c for c in cols_to_show if c in results.columns]
    display(results[cols_to_show].head(10))

    # Save
    save_path = f"{project_dir}/test_predictions.csv"
    results.to_csv(save_path, index=False)
    print(f"\nFull results saved to: {save_path}")
else:
    print(f"Error: Test CSV not found at {TEST_CSV}")

Loading test images from: /content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/data/sneakers/test
Starting inference on test set...


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]


Test Accuracy: 0.8462
Test AUC: 0.9545

===== SAMPLE PREDICTIONS =====


,Image Name,Brand,Authentic,Pred_Prob,Pred_Label
0,5.jpg,Adidas,0,0.132466,0
1,6.jpg,Adidas,0,0.102743,0
2,7.jpg,Adidas,0,0.119911,0
3,9.jpg,Jordan,1,0.909683,1
4,11.jpg,Nike,0,0.108156,0
5,12.jpg,Nike,0,0.121770,0
6,13.jpg,Nike,0,0.897639,1
7,14.jpg,Reebok,1,0.498992,0
8,2.jpg,Puma,0,0.012976,0
9,3.jpg,Puma,0,0.188564,0



Full results saved to: /content/drive/MyDrive/Masters/UIUC_FA2025/CS441/project/Counterfeit-Product-Image-Detection/test_predictions.csv
